In [ ]:
# Inputs: Centralize configuration for reuse
region         = boto3.Session().region_name 
bucket         = "your-s3-bucket" 
train_key      = "data/your-training.csv" 
target_col     = "target"
problem_type   = "BinaryClassification"
instance_type  = "ml.m5.large"
base_endpoint_name = "autopilot-poc-endpoint"  # base name for the endpoint

# Create a unique suffix for this deployment (timestamp or version)
suffix = time.strftime("%Y%m%d-%H%M%S")
auto_ml_job_name = f"autopilot-job-{suffix}"
model_name       = f"{auto_ml_job_name}-model"
endpoint_config_name = f"{base_endpoint_name}-config-{suffix}"
endpoint_name    = base_endpoint_name  # we will reuse the same endpoint name

# Launch Autopilot job (same as in step 1, now using variables defined above)
sm = boto3.client("sagemaker", region_name=region)
sm.create_auto_ml_job_v2(
    AutoMLJobName=auto_ml_job_name,
    AutoMLJobInputDataConfig=[{
        "ChannelType": "training",
        "ContentType": "text/csv;header=present",
        "DataSource": {"S3DataSource": {"S3DataType": "S3Prefix", "S3Uri": f"s3://{bucket}/{train_key}"}}
    }],
    OutputDataConfig={"S3OutputPath": f"s3://{bucket}/autopilot-output"},
    RoleArn=boto3.client("sts").get_caller_identity()["Arn"],
    AutoMLProblemTypeConfig={
        "TabularJobConfig": {
            "TargetAttributeName": target_col,
            "ProblemType": problem_type,
            "CompletionCriteria": {"MaxCandidates": 3}
        }
    }
)
print(f"Started AutoML job: {auto_ml_job_name}")
waiter = sm.get_waiter('auto_ml_job_complete') if 'auto_ml_job_complete' in dir(sm.waiter_names) else None
# (If SageMaker provided a waiter for Autopilot jobs, otherwise use a manual loop as above)
if waiter:
    waiter.wait(AutoMLJobName=auto_ml_job_name)
else:
    # Simple polling if waiter is not available
    status = "InProgress"
    while status == "InProgress":
        job = sm.describe_auto_ml_job_v2(AutoMLJobName=auto_ml_job_name)
        status = job["AutoMLJobStatus"]
        print(f"Job status: {status}")
        if status in ["Failed", "Stopped"]:
            raise RuntimeError(f"AutoML job ended with status: {status}")
        time.sleep(60)

# Create model from best candidate
job_desc = sm.describe_auto_ml_job_v2(AutoMLJobName=auto_ml_job_name)
best_candidate = job_desc["BestCandidate"]
sm.create_model(
    ModelName=model_name,
    Containers=best_candidate["InferenceContainers"],
    ExecutionRoleArn=boto3.client("iam").get_user()["User"].arn
)
print(f"Created model: {model_name}")

# Create endpoint config with the new model
sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[{
        "VariantName": "AllTraffic", "ModelName": model_name,
        "InstanceType": instance_type, "InitialInstanceCount": 1
    }]
)
print(f"Created endpoint config: {endpoint_config_name}")

# Deploy to the stable endpoint (update if it exists, otherwise create)
try:
    sm.describe_endpoint(EndpointName=endpoint_name)
    sm.update_endpoint(EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name)
    print(f"Updated endpoint: {endpoint_name} to new config {endpoint_config_name}")
except sm.exceptions.ClientError:
    sm.create_endpoint(EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name)
    print(f"Created new endpoint: {endpoint_name}")
waiter = sm.get_waiter("endpoint_in_service")
waiter.wait(EndpointName=endpoint_name)
print(f"Endpoint {endpoint_name} is InService (ready)")
